In [1]:
import pandas as pd
import numpy as np

In [15]:
path = r'C:\Users\HashiriNio\Desktop\BiLSTM-CRF-master\data\test.csv'
data = pd.read_csv(path)
data

,text,tag
0,将 军 百 战 死,B I B I S
1,将 军 队 带 上 战 场,S B I B I B I
2,将 将 军 带 上 战 车,S B I B I B I
3,军 战 将 军,S S B I
4,将 军 战 将 军,S S S B I
5,将 军 战 将 军,B I S B I


In [16]:
data['text'] = data['text'].apply(lambda x:x +' <end>')
data['tag'] = data['tag'].apply(lambda x:x + ' END')
data

,text,tag
0,将 军 百 战 死 <end>,B I B I S END
1,将 军 队 带 上 战 场 <end>,S B I B I B I END
2,将 将 军 带 上 战 车 <end>,S B I B I B I END
3,军 战 将 军 <end>,S S B I END
4,将 军 战 将 军 <end>,S S S B I END
5,将 军 战 将 军 <end>,B I S B I END


In [17]:
myDf = data
wordIndexDict={"<pad>":0}
wi=1
for row in myDf["text"].values.tolist():
   # if type(row)==float:
    #    print(row)
     #   break
    for word in row.split(" "):
        if word not in wordIndexDict:
            wordIndexDict[word]=wi
            wi+=1
vocabSize=wi


In [18]:
wordIndexDict

{'<pad>': 0,
 '将': 1,
 '军': 2,
 '百': 3,
 '战': 4,
 '死': 5,
 '<end>': 6,
 '队': 7,
 '带': 8,
 '上': 9,
 '场': 10,
 '车': 11}

In [19]:
import re

myDf["tag"]=myDf["tag"].apply(lambda x:re.sub("\-\S+","",x))
myDf

,text,tag
0,将 军 百 战 死 <end>,B I B I S END
1,将 军 队 带 上 战 场 <end>,S B I B I B I END
2,将 将 军 带 上 战 车 <end>,S B I B I B I END
3,军 战 将 军 <end>,S S B I END
4,将 军 战 将 军 <end>,S S S B I END
5,将 军 战 将 军 <end>,B I S B I END


In [20]:
import tqdm
maxLen = 8
tagIndexDict = {"PAD": 0}
ti = 1
for row in tqdm.tqdm(myDf["tag"].values.tolist()):
    for tag in row.split(" "):
        if tag not in tagIndexDict:
            tagIndexDict[tag] = ti
            ti += 1
tagSum = len(list(tagIndexDict.keys()))
myDf["tag"] = myDf["tag"].apply(lambda x:x.split()+["PAD" for i in range(maxLen-len(x.split()))])
myDf["tag"] = myDf["tag"].apply(lambda x:[tagIndexDict[tagItem] for tagItem in x])
# myDf["tag"] = myDf["tag"].apply(lambda x: [[0 if tagI != tagIndexDict[tagItem] else 1
#                                             for tagI in range(len(tagIndexDict))]
#                                             for tagItem in x])
y=np.array(myDf["tag"].values.tolist())
y

100%|██████████| 6/6 [00:00<?, ?it/s]


array([[1, 2, 1, 2, 3, 4, 0, 0],
       [3, 1, 2, 1, 2, 1, 2, 4],
       [3, 1, 2, 1, 2, 1, 2, 4],
       [3, 3, 1, 2, 4, 0, 0, 0],
       [3, 3, 3, 1, 2, 4, 0, 0],
       [1, 2, 3, 1, 2, 4, 0, 0]])

In [21]:
y.shape

(6, 8)

In [26]:
maxLen=max(len(row) for row in myDf["text"].values.tolist())


['将 军 百 战 死 <end>',
 '将 军 队 带 上 战 场 <end>',
 '将 将 军 带 上 战 车 <end>',
 '军 战 将 军 <end>',
 '将 军 战 将 军 <end>',
 '将 军 战 将 军 <end>']

In [23]:
maxLen

19

In [27]:
len(myDf["text"].values.tolist()[2])

19

In [35]:
a = np.array([
              [
                  [1, 5, 5, 2],
                  [9, -6, 2, 8],
                  [-3, 7, -9, 1]
              ],
 
              [
                  [-1, 7, -5, 3],
                  [9, 6, 2, 8],
                  [3, 7, 9, 1]
              ],
            [
                  [21, 6, -5, 2],
                  [9, 36, 2, 8],
                  [3, 7, 79, 1]
              ]
            ])

In [37]:
b = np.argmax(a,2)
b

array([[1, 0, 1],
       [1, 0, 2],
       [0, 1, 2]], dtype=int64)

In [42]:
import tensorflow as tf
x = tf.constant([1.,2.,5.,1.])
tf.nn.softmax(x)

<tf.Tensor: id=3, shape=(4,), dtype=float32, numpy=array([0.01685873, 0.04582679, 0.9204557 , 0.01685873], dtype=float32)>